In [1]:
#The only issue was the return filtered_key_val
# one way to fix is to simply remove this line
# second method is to encapsulate the second for loop in a function definition as you can see below 
from collections import defaultdict

text_file = sc.textFile("/FileStore/tables/Assignment_1/Question_1.txt")
counts = text_file.flatMap(lambda line: line.split(" ")).map(lambda word:(word, 1)).collect()
key_val = defaultdict(int)
for item in counts:
  key = item[0]
  val = item[1]
  key_val[key] += int(val)

## make it into a function so that way the return code works
def test (key_val):
  filtered_key_val = dict()
  for k, v in key_val.items():
    if v >= 100:
      filtered_key_val[k] = v
  return filtered_key_val

filtered_key_val=test(key_val)



##### 1. How can we preprocess the textual contents in the files? 

*First step is to load the csv using spark.read.csv then create an RDD. *
*Second, use the higher order function map and use lambda to select the Body column -> column 8. *
*Third, filter out None or null rows, and only take the rows that has < p > tag. *
*Lastly, strip out all the puncioations and noise from each line then split by space*

##### 2. What are the most frequent keywords in the textual contents of each programming language? 

*After cleaning the data from noise we will have a list of words, now simply convert it to key,val pairs using the map function.*
*Then, use reduceByKey() function to calculate the frequency of words*

##### 3. What percentage of questions in each programming language has accepted answers?
*Create another RDD and use AcceptedAnswerId column inorder to find the percentage. Simply count the rows that are digits and divide it by the total count of the RDD*

##### 4. What types of questions are asked for each programming languages?
Thers is a column called title which describes the question asked. First, create an rdd for column title, then filter to check if theres a why,what, or how in the title, else the question is considered other

In [3]:
import re
##Loading java.csv
java_df=spark.read.csv("/FileStore/tables/Assignment_1/SO_Java.csv",inferSchema=True,header=True,multiLine=True,escape='"')
java_RDD=java_df.select('Body').rdd.flatMap(list)

## preprocess the textual content
java_RDD_body=java_RDD.filter(lambda x: x is not None)
java_RDD_body=java_RDD_body.filter(lambda x: '<p>' in x)
java_filtered=java_RDD_body.map(lambda x: re.sub(r'[^\w]', ' ', x))
java_words=java_filtered.flatMap(lambda line: line.split()).filter(lambda x: len(x)>2).filter(lambda x: not x.isdigit())
#java_words=java_words.filter((lambda x: x is not '=' ))
#java_words=java_words.filter((lambda x: x is not '{' ))
#java_words=java_words.filter((lambda x: x is not '}' ))
#java_words=java_words.filter((lambda x: x is not '+' ))
#java_words=java_words.filter((lambda x: '</code></pre>' not in x ))

## Creating key,val pair for each word then counting freq 
java_pairs=java_words.map(lambda x: (x,1))
java_freq=java_pairs.reduceByKey(lambda x,y: (x+y)).map(lambda x: (x[1],x[0])).sortByKey(ascending=False).collect()

##calculating % of answered questions
java_accedpted_df=java_df.select('AcceptedAnswerId')
java_total_count=java_accedpted_df.count()
java_accedpted_numeric_df=java_accedpted_df.filter(java_accedpted_df.AcceptedAnswerId >0)
java_count_of_numeric=java_accedpted_numeric_df.count()

## Findindin types of question
java_RDD_title=java_df.select('Title','Id').rdd
java_RDD_title=java_RDD_title.map(lambda x: (x[1],x[0]))
java_RDD_title=java_RDD_title.map(lambda x:(x[0],x[1].split('?')))
java_RDD_title=java_RDD_title.map(lambda x:(x[0],list(filter(None, x[1]))))
java_RDD_title=java_RDD_title.map(lambda x:(x[0], re.sub(r'[^\w]', ' ', x[1][0])))

java_what=java_RDD_title.map(lambda x:(x[0],x[1].split())).filter(lambda x : ('what' in x[1] or 'What' in x[1] or 'whats' in x[1] or 'Whats' in x[1])) 
java_what=java_what.filter(lambda x : ('how' not in [1] and 'How' not in [1]) or ('what' in [1][0] or 'What'in x[1][0] or 'whats' in [1][0] or 'Whats'in x[1][0]) ) 
java_what=java_what.filter(lambda x : ('why' not in [1] and 'Why' not in [1])or ('what' in [1][0] or 'What'in x[1][0] or 'whats' in [1][0] or 'Whats'in x[1][0])).map(lambda x: (x[0]," ".join(x[1])))

java_why=java_RDD_title.map(lambda x: (x[0],x[1].split())).filter(lambda x : ('why' in x[1] or 'Why'in x[1])) 
java_why=java_why.filter(lambda x : ('how' not in x[1] and 'How' not in x[1])or ('why' in x[1][0] or 'Why'in x[1][0])) 
java_why=java_why.filter(lambda x : ('what' not in x[1] and 'What' not in x[1]) or ('why' in x[1][0] or 'Why'in x[1][0])).map(lambda x:(x[0]," ".join(x[1])))

java_how=java_RDD_title.map(lambda x: (x[0],x[1].split())).filter(lambda x : ('how' in x[1] or 'How'in x[1])) 
java_how=java_how.filter(lambda x : ('what' not in x[1] and 'What' not in x[1]) or ('how' in x[1][0] or 'How'in x[1][0])) 
java_how=java_how.filter(lambda x : ('why' not in x[1] and 'Why' not in x[1])or ('how' in x[1][0] or 'How'in x[1][0])).map(lambda x:(x[0]," ".join(x[1])))

java_combined=java_what.union(java_why)
java_combined=java_how.union(java_combined)
java_other=java_RDD_title.subtractByKey(java_combined)

java_what=java_what.map(lambda x: x[1]).count()
java_why=java_why.map(lambda x: x[1]).count()
java_how=java_how.map(lambda x: x[1]).count()
java_other=java_other.map(lambda x: x[1]).count()

In [4]:
print("Most frequent words in Java are")
print(java_freq[:10])
print("Percentage of answered questions in Java is: {:.2f}%".format(java_count_of_numeric/java_total_count*100))
print("Types of question in Java What={}, Why={}, How={}, and Other={}".format(java_what,java_why,java_how,java_other))


Most frequent words in Java are
[(288913, 'code'), (264917, 'the'), (163381, 'java'), (153397, 'pre'), (95536, 'new'), (95041, 'public'), (94912, 'and'), (86852, 'this'), (84927, 'org'), (80600, 'String')]
Percentage of answered questions in Java is: 50.09%
Types of question in Java What=1190, Why=1489, How=10247, and Other=37098

In [5]:
import re
##Loading py.csv
py_df=spark.read.csv("/FileStore/tables/Assignment_1/SO_Python.csv",inferSchema=True,header=True,multiLine=True,escape='"')
py_RDD=py_df.select('Body').rdd.flatMap(list)

## preprocess the textual content
py_RDD_body=py_RDD.filter(lambda x: x is not None)
py_RDD_body=py_RDD_body.filter(lambda x: '<p>' in x)
py_filtered=py_RDD_body.map(lambda x: re.sub(r'[^\w]', ' ', x))
py_words=py_filtered.flatMap(lambda line: line.split()).filter(lambda x: len(x)>2).filter(lambda x: not x.isdigit())
#py_words=py_words.filter((lambda x: x is not '=' ))
#py_words=py_words.filter((lambda x: x is not '{' ))
#py_words=py_words.filter((lambda x: x is not '}' ))
#py_words=py_words.filter((lambda x: x is not '+' ))
#py_words=py_words.filter((lambda x: '</code></pre>' not in x ))

## Creating key,val pair for each word then counting freq 
py_pairs=py_words.map(lambda x: (x,1))
py_freq=py_pairs.reduceByKey(lambda x,y: (x+y)).map(lambda x: (x[1],x[0])).sortByKey(ascending=False).collect()

##calculating % of answered questions
py_accedpted_df=py_df.select('AcceptedAnswerId')
py_total_count=py_accedpted_df.count()
py_accedpted_numeric_df=py_accedpted_df.filter(py_accedpted_df.AcceptedAnswerId >0)
py_count_of_numeric=py_accedpted_numeric_df.count()

## Findindin types of question
py_RDD_title=py_df.select('Title','Id').rdd
py_RDD_title=py_RDD_title.map(lambda x: (x[1],x[0]))
py_RDD_title=py_RDD_title.map(lambda x:(x[0],x[1].split('?')))
py_RDD_title=py_RDD_title.map(lambda x:(x[0],list(filter(None, x[1]))))
py_RDD_title=py_RDD_title.map(lambda x:(x[0], re.sub(r'[^\w]', ' ', x[1][0])))

py_what=py_RDD_title.map(lambda x:(x[0],x[1].split())).filter(lambda x : ('what' in x[1] or 'What' in x[1] or 'whats' in x[1] or 'Whats' in x[1])) 
py_what=py_what.filter(lambda x : ('how' not in [1] and 'How' not in [1]) or ('what' in [1][0] or 'What'in x[1][0] or 'whats' in [1][0] or 'Whats'in x[1][0]) ) 
py_what=py_what.filter(lambda x : ('why' not in [1] and 'Why' not in [1])or ('what' in [1][0] or 'What'in x[1][0] or 'whats' in [1][0] or 'Whats'in x[1][0])).map(lambda x: (x[0]," ".join(x[1])))

py_why=py_RDD_title.map(lambda x: (x[0],x[1].split())).filter(lambda x : ('why' in x[1] or 'Why'in x[1])) 
py_why=py_why.filter(lambda x : ('how' not in x[1] and 'How' not in x[1])or ('why' in x[1][0] or 'Why'in x[1][0])) 
py_why=py_why.filter(lambda x : ('what' not in x[1] and 'What' not in x[1]) or ('why' in x[1][0] or 'Why'in x[1][0])).map(lambda x:(x[0]," ".join(x[1])))

py_how=py_RDD_title.map(lambda x: (x[0],x[1].split())).filter(lambda x : ('how' in x[1] or 'How'in x[1])) 
py_how=py_how.filter(lambda x : ('what' not in x[1] and 'What' not in x[1]) or ('how' in x[1][0] or 'How'in x[1][0])) 
py_how=py_how.filter(lambda x : ('why' not in x[1] and 'Why' not in x[1])or ('how' in x[1][0] or 'How'in x[1][0])).map(lambda x:(x[0]," ".join(x[1])))

py_combined=py_what.union(py_why)
py_combined=py_how.union(py_combined)
py_other=py_RDD_title.subtractByKey(py_combined)

py_what=py_what.map(lambda x: x[1]).count()
py_why=py_why.map(lambda x: x[1]).count()
py_how=py_how.map(lambda x: x[1]).count()
py_other=py_other.map(lambda x: x[1]).count()


In [6]:
print("Most frequent words in Python are")
print(py_freq[:10])
print("Percentage of answered questions in Python is: {:.2f}%".format(py_count_of_numeric/py_total_count*100))
print("Types of question in Python What={}, Why={}, How={}, and Other={}".format(py_what,py_why,py_how,py_other))


Most frequent words in Python are
[(326057, 'code'), (259933, 'the'), (173034, 'pre'), (114246, 'quot'), (97612, 'and'), (82290, 'self'), (74034, 'for'), (59803, 'this'), (52797, 'that'), (48419, 'from')]
Percentage of answered questions in Python is: 51.10%
Types of question in Python What=860, Why=1251, How=10997, and Other=36921

In [7]:
import re
##Loading js.csv
js_df=spark.read.csv("/FileStore/tables/Assignment_1/SO_Javascript.csv",inferSchema=True,header=True,multiLine=True,escape='"')
js_RDD=js_df.select('Body').rdd.flatMap(list)

## preprocess the textual content
js_RDD_body=js_RDD.filter(lambda x: x is not None)
js_RDD_body=js_RDD_body.filter(lambda x: '<p>' in x)
js_filtered=js_RDD_body.map(lambda x: re.sub(r'[^\w]', ' ', x))
js_words=js_filtered.flatMap(lambda line: line.split()).filter(lambda x: len(x)>2).filter(lambda x: not x.isdigit())
#js_words=js_words.filter((lambda x: x is not '=' ))
#js_words=js_words.filter((lambda x: x is not '{' ))
#js_words=js_words.filter((lambda x: x is not '}' ))
#js_words=js_words.filter((lambda x: x is not '+' ))
#js_words=js_words.filter((lambda x: '</code></pre>' not in x ))

## Creating key,val pair for each word then counting freq 
js_pairs=js_words.map(lambda x: (x,1))
js_freq=js_pairs.reduceByKey(lambda x,y: (x+y)).map(lambda x: (x[1],x[0])).sortByKey(ascending=False).collect()

##calculating % of answered questions
js_accedpted_df=js_df.select('AcceptedAnswerId')
js_total_count=js_accedpted_df.count()
js_accedpted_numeric_df=js_accedpted_df.filter(js_accedpted_df.AcceptedAnswerId >0)
js_count_of_numeric=js_accedpted_numeric_df.count()

## Findindin types of question
js_RDD_title=js_df.select('Title','Id').rdd
js_RDD_title=js_RDD_title.map(lambda x: (x[1],x[0]))
js_RDD_title=js_RDD_title.map(lambda x:(x[0],x[1].split('?')))
js_RDD_title=js_RDD_title.map(lambda x:(x[0],list(filter(None, x[1]))))
js_RDD_title=js_RDD_title.map(lambda x:(x[0], re.sub(r'[^\w]', ' ', x[1][0])))

js_what=js_RDD_title.map(lambda x:(x[0],x[1].split())).filter(lambda x : ('what' in x[1] or 'What' in x[1] or 'whats' in x[1] or 'Whats' in x[1])) 
js_what=js_what.filter(lambda x : ('how' not in [1] and 'How' not in [1]) or ('what' in [1][0] or 'What'in x[1][0] or 'whats' in [1][0] or 'Whats'in x[1][0]) ) 
js_what=js_what.filter(lambda x : ('why' not in [1] and 'Why' not in [1])or ('what' in [1][0] or 'What'in x[1][0] or 'whats' in [1][0] or 'Whats'in x[1][0])).map(lambda x: (x[0]," ".join(x[1])))

js_why=js_RDD_title.map(lambda x: (x[0],x[1].split())).filter(lambda x : ('why' in x[1] or 'Why'in x[1])) 
js_why=js_why.filter(lambda x : ('how' not in x[1] and 'How' not in x[1])or ('why' in x[1][0] or 'Why'in x[1][0])) 
js_why=js_why.filter(lambda x : ('what' not in x[1] and 'What' not in x[1]) or ('why' in x[1][0] or 'Why'in x[1][0])).map(lambda x:(x[0]," ".join(x[1])))

js_how=js_RDD_title.map(lambda x: (x[0],x[1].split())).filter(lambda x : ('how' in x[1] or 'How'in x[1])) 
js_how=js_how.filter(lambda x : ('what' not in x[1] and 'What' not in x[1]) or ('how' in x[1][0] or 'How'in x[1][0])) 
js_how=js_how.filter(lambda x : ('why' not in x[1] and 'Why' not in x[1])or ('how' in x[1][0] or 'How'in x[1][0])).map(lambda x:(x[0]," ".join(x[1])))

js_combined=js_what.union(js_why)
js_combined=js_how.union(js_combined)
js_other=js_RDD_title.subtractByKey(js_combined)

js_what=js_what.map(lambda x: x[1]).count()
js_why=js_why.map(lambda x: x[1]).count()
js_how=js_how.map(lambda x: x[1]).count()
js_other=js_other.map(lambda x: x[1]).count()


In [8]:
print("Most frequent words in JavaScript are")
print(js_freq[:10])
print("Percentage of answered questions in JavaScript is: {:.2f}%".format(js_count_of_numeric/js_total_count*100))
print("Types of question in JavaScript What={}, Why={}, How={}, and Other={}".format(js_what,js_why,js_how,js_other))
